In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy

import torch.nn.init as init

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())


PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('/home/c/cl237/', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='/home/c/cl237/', train=False, download=True, transform=ts)

In [3]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [4]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)

#torch.empty() return a tensor filled with garbage values Dimention: N by x_d0*x_d1*x_d2

for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [5]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [6]:
#### Chenyang: you need to write a function to transform the x_trainTensor, x_testTensor to (H'W')*(l^2C)*n
#### follow the paper in https://arxiv.org/pdf/1611.03214.pdf   this paper donot have the stride, we should also include stride
#### how to make it faster;  make sure the code is correct?
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [7]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [8]:
#### To-do-list
#### Chenyang: you need to write a function to update W for CNN
def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    # _, col_U = U_prime.size()
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    # bstar = 0
    return Wstar
# def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W, v for the first layer should be X-bar-bar-prime
#     W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
#     W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
#     d,N = V.size()
#     I = torch.eye(N, device=device)
#     U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
#     A = rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix
#     A = torch.t(A)
#     # _, col_U = U_prime.size()
#     Wstar_transpose = torch.mm(A, torch.inverse(rho*(torch.mm(torch.t(V), V))+(alpha+tau)*I))
#     Wstar =  torch.t(Wstar_transpose)
#     return Wstar

def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
niter = 2000
rank = 220
tau = 0.2
gamma = 0.5
alpha = 1
rho = 0.5

print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)

S = 32 ### number of filters 2^5
H, W, C, n = x_trainTensor.size()   # n is the same thing as N
Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
Hprime = Hprime.to(torch.int)
Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
Wprime = Wprime.to(torch.int)



for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    ## Chenyang
    d1 =  Hprime * Wprime * S
    d2 =  1024
    d3 =  1024
    d4 =  10


    W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  ## How people usually initialize the CNN kernel?
    #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2)) ## TBD
    #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    #factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, 1))
    #W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    limit = torch.sqrt(torch.tensor(3. / d1))
    W2 = torch.empty(d2, d1, device=device).uniform_(-limit, limit)
    W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) # 2^10 and Hprime * Wprime  *2^5*1^3
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray
    b2 = 0*torch.ones(d2, 1, device=device)


    limit = torch.sqrt(torch.tensor(3. / d2))
    W3 = torch.empty(d3, d2, device=device).uniform_(-limit, limit)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    limit = torch.sqrt(torch.tensor(3. / d3))
    W4 = torch.empty(d4, d3, device=device).uniform_(-limit, limit)
    b4 = 0*torch.ones(d4, 1, device=device)

    ## Chenyang: this is for the initialization of U1
    x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
    x_trainTS1 = torch.t(x_trainTS1)
    x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar'
    U1prime = torch.matmul(x_trainTS1, W1)
    U1prime = torch.reshape(U1prime, (n,-1))
    U1 = torch.t(U1prime)

    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = U4

    ### to create the X-bar-bar' for test data
    x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
    x_testTS1 = torch.t(x_testTS1)
    x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar' test


    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

  # update for last layer
        # update V4
        V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

        # update U4
        U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


 # update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        ## Chenyang
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        # update U1
        U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        ## Chenyang: you need to use the new function updateWb_CNN
        W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)

        # G update
        #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2))
        #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        #factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,1))
        #W1_tl_tensor_rec = tt_to_tensor(factors1)




  # prediction for trainning data
        ### we should use reconstructed tensnor W1 for prediction
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        a1_train = nn.ReLU()(XprimeWtranspose)
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)
        # check argmax and addmm, dim=0

  #Prediction for test data
        XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
        XprimeWtesttranspose = torch.t(XprimeWtest)
        a1_test = nn.ReLU()(XprimeWtesttranspose) ## Chenyang
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


    #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

        # Eq (5) in paper
        ## Chenyang
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy()
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy()

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    # factors1_shape=[f.shape for f in factors1]
    # Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3

    layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
    layer3_CR = Sum_of_variables_factors3/(d2*d3)
    Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
    Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()

    # print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)

    #this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
    #new_row = {'rank':rank, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'layer2_CR':layer2_CR, \
    #          'layer3_CR':layer3_CR, 'Compressedlayers_CR':Compressedlayers_CR}
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv',index=False)


filename="LecunUniform_"+ "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCDCNN_mnist_init
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})


rank= 220 tau= 0.2 gamma= 0.5 rho= 0.5 alpha 1


/home/c/cl237/.conda/envs/torch-cuda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 5.93969464302063 - sq_loss: 541.394775390625 - tot_loss: 741.620980779163 - acc: 0.238 - val_acc: 0.24124124124124124
Repeatition 1 Epoch 2 / 2000 
 - time: 5.649649381637573 - sq_loss: 240.61993408203125 - tot_loss: 416.389953542687 - acc: 0.3955 - val_acc: 0.3973973973973974
Repeatition 1 Epoch 3 / 2000 
 - time: 5.64014196395874 - sq_loss: 122.81843566894531 - tot_loss: 243.20438879691065 - acc: 0.5078333333333334 - val_acc: 0.5035035035035035
Repeatition 1 Epoch 4 / 2000 
 - time: 5.635732650756836 - sq_loss: 65.19322204589844 - tot_loss: 144.82061434630305 - acc: 0.5838333333333333 - val_acc: 0.5665665665665666
Repeatition 1 Epoch 5 / 2000 
 - time: 5.603754997253418 - sq_loss: 35.03505325317383 - tot_loss: 88.48441620878876 - acc: 0.6338333333333334 - val_acc: 0.6276276276276276
Repeatition 1 Epoch 6 / 2000 
 - time: 5.60964822769165 - sq_loss: 18.92500877380371 - tot_loss: 55.75841420292854 - a

Repeatition 1 Epoch 49 / 2000 
 - time: 5.635284185409546 - sq_loss: 0.00026132730999961495 - tot_loss: 0.4748665457474999 - acc: 0.8971666666666667 - val_acc: 0.8908908908908909
Repeatition 1 Epoch 50 / 2000 
 - time: 5.570619821548462 - sq_loss: 0.00024591025430709124 - tot_loss: 0.46418250355636703 - acc: 0.898 - val_acc: 0.8928928928928929
Repeatition 1 Epoch 51 / 2000 
 - time: 5.593944072723389 - sq_loss: 0.00023204641183838248 - tot_loss: 0.45402772672241554 - acc: 0.8988333333333334 - val_acc: 0.8928928928928929
Repeatition 1 Epoch 52 / 2000 
 - time: 5.60301947593689 - sq_loss: 0.00021957681747153401 - tot_loss: 0.444360856001731 - acc: 0.9 - val_acc: 0.8958958958958959
Repeatition 1 Epoch 53 / 2000 
 - time: 5.573245048522949 - sq_loss: 0.0002083219587802887 - tot_loss: 0.43514618285698814 - acc: 0.9011666666666667 - val_acc: 0.8968968968968969
Repeatition 1 Epoch 54 / 2000 
 - time: 5.59078311920166 - sq_loss: 0.00019817016436718404 - tot_loss: 0.4263504060509149 - acc: 0.90

Repeatition 1 Epoch 96 / 2000 
 - time: 5.832059383392334 - sq_loss: 7.255177479237318e-05 - tot_loss: 0.23656986315327233 - acc: 0.925 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 97 / 2000 
 - time: 5.788743257522583 - sq_loss: 7.157379150157794e-05 - tot_loss: 0.2341444820704055 - acc: 0.925 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 98 / 2000 
 - time: 5.810433864593506 - sq_loss: 7.062472286634147e-05 - tot_loss: 0.23176892726041842 - acc: 0.9253333333333333 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 99 / 2000 
 - time: 5.773969411849976 - sq_loss: 6.970766116864979e-05 - tot_loss: 0.22944156397716142 - acc: 0.9258333333333333 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 100 / 2000 
 - time: 5.801197290420532 - sq_loss: 6.881114677526057e-05 - tot_loss: 0.22716115434304812 - acc: 0.9263333333333333 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 101 / 2000 
 - time: 5.8212058544158936 - sq_loss: 6.793138891225681e-05 - tot_loss: 0.22492616504459875 - acc:

...

Repeatition 1 Epoch 1787 / 2000 
 - time: 6.616398572921753 - sq_loss: 1.8966766219818965e-06 - tot_loss: 0.01675332574541244 - acc: 0.9925 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1788 / 2000 
 - time: 6.618377208709717 - sq_loss: 1.8946202544611879e-06 - tot_loss: 0.016744798377840195 - acc: 0.9925 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1789 / 2000 
 - time: 6.5892014503479 - sq_loss: 1.8932161083284882e-06 - tot_loss: 0.016736279151359668 - acc: 0.9925 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1790 / 2000 
 - time: 6.673919677734375 - sq_loss: 1.892005343506753e-06 - tot_loss: 0.016727764001893772 - acc: 0.9925 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1791 / 2000 
 - time: 6.6080896854400635 - sq_loss: 1.8907338699136744e-06 - tot_loss: 0.016719252575853716 - acc: 0.9925 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1792 / 2000 
 - time: 6.622232675552368 - sq_loss: 1.8889884358941345e-06 - tot_loss: 0.016710750965057742 - acc: 0.9925 - val_acc: 0

Repeatition 1 Epoch 1834 / 2000 
 - time: 6.662306070327759 - sq_loss: 1.8315830629944685e-06 - tot_loss: 0.016361279474108415 - acc: 0.9928333333333333 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1835 / 2000 
 - time: 6.628122806549072 - sq_loss: 1.8304762079424108e-06 - tot_loss: 0.01635313241829408 - acc: 0.9928333333333333 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1836 / 2000 
 - time: 6.661031723022461 - sq_loss: 1.8291127616976155e-06 - tot_loss: 0.016344992660879146 - acc: 0.9928333333333333 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1837 / 2000 
 - time: 6.641249895095825 - sq_loss: 1.827949517974048e-06 - tot_loss: 0.016336853747861823 - acc: 0.9928333333333333 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1838 / 2000 
 - time: 6.595174789428711 - sq_loss: 1.8265772041559103e-06 - tot_loss: 0.016328723469246145 - acc: 0.9928333333333333 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1839 / 2000 
 - time: 6.645868539810181 - sq_loss: 1.8253524558531353e-0

Repeatition 1 Epoch 1882 / 2000 
 - time: 6.613264799118042 - sq_loss: 1.7711290638544597e-06 - tot_loss: 0.0159789138364431 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1883 / 2000 
 - time: 6.600350379943848 - sq_loss: 1.7696100940156612e-06 - tot_loss: 0.015971129550121078 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1884 / 2000 
 - time: 6.64818549156189 - sq_loss: 1.7685166540104547e-06 - tot_loss: 0.015963352928633868 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1885 / 2000 
 - time: 6.631619453430176 - sq_loss: 1.7673721686151111e-06 - tot_loss: 0.015955584293669743 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1886 / 2000 
 - time: 6.660515546798706 - sq_loss: 1.765949832588376e-06 - tot_loss: 0.015947828344246774 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1887 / 2000 
 - time: 6.618603706359863 - sq_loss: 1.7644847503106575e-06 

Repeatition 1 Epoch 1928 / 2000 
 - time: 6.675360202789307 - sq_loss: 1.7152785858343123e-06 - tot_loss: 0.015628675189827846 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1929 / 2000 
 - time: 6.623793840408325 - sq_loss: 1.714581571832241e-06 - tot_loss: 0.015621215794158162 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1930 / 2000 
 - time: 6.695793390274048 - sq_loss: 1.7131163758676848e-06 - tot_loss: 0.015613770417417072 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1931 / 2000 
 - time: 6.676262617111206 - sq_loss: 1.7120834172601462e-06 - tot_loss: 0.015606329073602866 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1932 / 2000 
 - time: 6.623883247375488 - sq_loss: 1.7108809515775647e-06 - tot_loss: 0.015598895443872607 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1933 / 2000 
 - time: 6.649704217910767 - sq_loss: 1.7097198679039138e-

Repeatition 1 Epoch 1975 / 2000 
 - time: 6.610360860824585 - sq_loss: 1.6601081824774155e-06 - tot_loss: 0.015286069011881408 - acc: 0.9935 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1976 / 2000 
 - time: 6.642834424972534 - sq_loss: 1.6587167692705407e-06 - tot_loss: 0.015278951129471353 - acc: 0.9935 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1977 / 2000 
 - time: 6.676146030426025 - sq_loss: 1.6573463881286443e-06 - tot_loss: 0.01527184255503471 - acc: 0.9935 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1978 / 2000 
 - time: 6.637120962142944 - sq_loss: 1.6562959217480966e-06 - tot_loss: 0.015264733266872098 - acc: 0.9935 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1979 / 2000 
 - time: 6.707690238952637 - sq_loss: 1.6550328609810094e-06 - tot_loss: 0.015257631347844835 - acc: 0.9935 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1980 / 2000 
 - time: 6.651038646697998 - sq_loss: 1.6539478338017943e-06 - tot_loss: 0.01525053742068394 - acc: 0.9935 - val_acc: 